In [ ]:
!pip install pandas sqlite3


ERROR: Could not find a version that satisfies the requirement sqlite3 (from versions: none)
ERROR: No matching distribution found for sqlite3


In [ ]:
import pandas as pd
import sqlite3


In [ ]:
df = pd.read_csv('/content/flights_sample_3m.csv')
df


,FL_DATE,AIRLINE,AIRLINE_DOT,AIRLINE_CODE,DOT_CODE,FL_NUMBER,ORIGIN,ORIGIN_CITY,DEST,DEST_CITY,...,DIVERTED,CRS_ELAPSED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,DELAY_DUE_CARRIER,DELAY_DUE_WEATHER,DELAY_DUE_NAS,DELAY_DUE_SECURITY,DELAY_DUE_LATE_AIRCRAFT
0,2019-01-09,United Air Lines Inc.,United Air Lines Inc.: UA,UA,19977,1562,FLL,"Fort Lauderdale, FL",EWR,"Newark, NJ",...,0.0,186.0,176.0,153.0,1065.0,NaN,NaN,NaN,NaN,NaN
1,2022-11-19,Delta Air Lines Inc.,Delta Air Lines Inc.: DL,DL,19790,1149,MSP,"Minneapolis, MN",SEA,"Seattle, WA",...,0.0,235.0,236.0,189.0,1399.0,NaN,NaN,NaN,NaN,NaN
2,2022-07-22,United Air Lines Inc.,United Air Lines Inc.: UA,UA,19977,459,DEN,"Denver, CO",MSP,"Minneapolis, MN",...,0.0,118.0,112.0,87.0,680.0,NaN,NaN,NaN,NaN,NaN
3,2023-03-06,Delta Air Lines Inc.,Delta Air Lines Inc.: DL,DL,19790,2295,MSP,"Minneapolis, MN",SFO,"San Francisco, CA",...,0.0,260.0,285.0,249.0,1589.0,0.0,0.0,24.0,0.0,0.0
4,2020-02-23,Spirit Air Lines,Spirit Air Lines: NK,NK,20416,407,MCO,"Orlando, FL",DFW,"Dallas/Fort Worth, TX",...,0.0,181.0,182.0,153.0,985.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2999995,2022-11-13,American Airlines Inc.,American Airlines Inc.: AA,AA,19805,1522,JAX,"Jacksonville, FL",CLT,"Charlotte, NC",...,0.0,85.0,71.0,55.0,328.0,NaN,NaN,NaN,NaN,NaN
2999996,2022-11-02,American Airlines Inc.,American Airlines Inc.: AA,AA,19805,1535,ORD,"Chicago, IL",AUS,"Austin, TX",...,0.0,176.0,145.0,130.0,977.0,NaN,NaN,NaN,NaN,NaN
2999997,2022-09-11,Delta Air Lines Inc.,Delta Air Lines Inc.: DL,DL,19790,2745,HSV,"Huntsville, AL",ATL,"Atlanta, GA",...,0.0,55.0,50.0,28.0,151.0,0.0,36.0,0.0,0.0,0.0
2999998,2019-11-13,Republic Airline,Republic Airline: YX,YX,20452,6134,BOS,"Boston, MA",LGA,"New York, NY",...,0.0,88.0,77.0,50.0,184.0,NaN,NaN,NaN,NaN,NaN


In [ ]:
df.info()
df.isnull().sum()  # Count missing values


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000000 entries, 0 to 2999999
Data columns (total 32 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   FL_DATE                  object 
 1   AIRLINE                  object 
 2   AIRLINE_DOT              object 
 3   AIRLINE_CODE             object 
 4   DOT_CODE                 int64  
 5   FL_NUMBER                int64  
 6   ORIGIN                   object 
 7   ORIGIN_CITY              object 
 8   DEST                     object 
 9   DEST_CITY                object 
 10  CRS_DEP_TIME             int64  
 11  DEP_TIME                 float64
 12  DEP_DELAY                float64
 13  TAXI_OUT                 float64
 14  WHEELS_OFF               float64
 15  WHEELS_ON                float64
 16  TAXI_IN                  float64
 17  CRS_ARR_TIME             int64  
 18  ARR_TIME                 float64
 19  ARR_DELAY                float64
 20  CANCELLED                float64
 21  CANCELLA

,0
FL_DATE,0
AIRLINE,0
AIRLINE_DOT,0
AIRLINE_CODE,0
DOT_CODE,0
FL_NUMBER,0
ORIGIN,0
ORIGIN_CITY,0
DEST,0
DEST_CITY,0


In [ ]:
conn = sqlite3.connect(':memory:')  # Creates a temporary SQLite database


In [ ]:
df.to_sql('flight_data', conn, index=False, if_exists='replace')  # Save DataFrame as SQL table


3000000

# **Total Number of Flights per Year**

In [ ]:
query = '''
SELECT strftime('%Y', FL_DATE) as Year, COUNT(*) as Total_Flights
FROM flight_data
GROUP BY year;
'''
result= pd.read_sql_query(query, conn)
result

,Year,Total_Flights
0,2019,757673
1,2020,479350
2,2021,611633
3,2022,687860
4,2023,463484


- This query retrieves the total number of flights for each year by extracting the year from the flight date and grouping the data accordingly.
- It helps to understand the annual flight volume trends.

In [ ]:
result.to_csv('result1.csv', index=False)

# **Total Cancellations by Year**

In [ ]:
query = '''
SELECT strftime('%Y', FL_DATE) as Year,
       COUNT(CASE WHEN CANCELLED=1 THEN 1 END) as Total_Cancellations,
       COUNT(*) as Total_Flights,
       ROUND((COUNT(CASE WHEN CANCELLED=1 THEN 1 END) * 100.0 / COUNT(*)),3) as Cancellation_Percentage
FROM flight_data
GROUP BY year;
'''
result= pd.read_sql_query(query, conn)
result

,Year,Total_Cancellations,Total_Flights,Cancellation_Percentage
0,2019,13594,757673,1.794
1,2020,28757,479350,5.999
2,2021,10532,611633,1.722
3,2022,18448,687860,2.682
4,2023,7809,463484,1.685


- This query calculates the total number of cancellations, total flights, and the percentage of cancellations for each year.
- It gives insights into how cancellations have varied over time and their proportion relative to total flights.

In [ ]:
result.to_csv('result2.csv', index=False)

# **On-Time vs Delayed Flights Yearly**

In [ ]:
query = '''
SELECT strftime('%Y', FL_DATE) as Year,
       COUNT(CASE WHEN DEP_DELAY <= 0 THEN 1 END) as On_Time_Flights,
       COUNT(CASE WHEN DEP_DELAY > 0 THEN 1 END) as Delayed_Flights,
       ROUND((COUNT(CASE WHEN DEP_DELAY <= 0 THEN 1 END) * 100.0 / COUNT(*)),3) as On_Time_Percentage,
       ROUND((COUNT(CASE WHEN DEP_DELAY > 0 THEN 1 END) * 100.0 / COUNT(*)),3) as Delayed_Percentage
FROM flight_data
GROUP BY year;
'''
result= pd.read_sql_query(query, conn)
result

,Year,On_Time_Flights,Delayed_Flights,On_Time_Percentage,Delayed_Percentage
0,2019,487494,257037,64.341,33.925
1,2020,364058,86649,75.948,18.076
2,2021,399254,202081,65.277,33.040
3,2022,407276,262563,59.209,38.171
4,2023,271431,184513,58.563,39.810


- This query identifies the number of on-time and delayed flights each year, along with their percentages.
- It highlights annual punctuality trends and the proportion of delayed flights.

In [ ]:
result.to_csv('result3.csv', index=False)

# **Average Delay Time by Year**

In [ ]:
query = '''
SELECT strftime('%Y', FL_DATE) as Year,
       AVG(DEP_DELAY) as Average_Departure_Delay,
       AVG(ARR_DELAY) as Average_Arrival_Delay
FROM flight_data
WHERE DEP_DELAY > 0 OR ARR_DELAY > 0
GROUP BY year;
'''
result= pd.read_sql_query(query, conn)
result

,Year,Average_Departure_Delay,Average_Arrival_Delay
0,2019,30.944973,30.114348
1,2020,21.970824,21.900776
2,2021,28.634240,26.795731
3,2022,31.294262,29.589928
4,2023,34.285028,33.370908


- This query calculates the average delay times for departures and arrivals annually.
- It excludes flights without delays, providing a focused view of delay durations.

In [ ]:
result.to_csv('result4.csv', index=False)

# **Impact of Scheduled Flight Time on Delays**

In [ ]:
query = '''
SELECT CRS_DEP_TIME,
       ROUND((COUNT(CASE WHEN DEP_DELAY > 0 THEN 1 END) * 100.0 / COUNT(*)),3) as Departure_Delay_Percentage,
       ROUND((COUNT(CASE WHEN ARR_DELAY > 0 THEN 1 END) * 100.0 / COUNT(*)),3) as Arrival_Delay_Percentage
FROM flight_data
GROUP BY CRS_DEP_TIME
ORDER BY CRS_DEP_TIME;
'''
result= pd.read_sql_query(query, conn)
result

,CRS_DEP_TIME,Departure_Delay_Percentage,Arrival_Delay_Percentage
0,1,54.167,54.167
1,2,40.000,20.000
2,3,100.000,100.000
3,4,58.824,41.176
4,5,38.971,25.000
...,...,...,...
1379,2355,36.000,31.875
1380,2356,40.580,27.053
1381,2357,49.198,41.711
1382,2358,47.143,36.190


- This query examines how scheduled departure times (CRS_DEP_TIME) influence the percentage of delays for departures and arrivals.
- It provides insights into delay patterns across different times of the day.


In [ ]:
result.to_csv('result5.csv', index=False)

# **Flight Delays by Airport**

In [ ]:
query = '''
SELECT ORIGIN,
       ORIGIN_CITY,
       ROUND((COUNT(CASE WHEN DEP_DELAY > 0 THEN 1 END) * 100.0 / COUNT(*)),3) as Departure_Delay_Percentage
FROM flight_data
GROUP BY ORIGIN
ORDER BY Departure_Delay_Percentage DESC
LIMIT 10;
'''
result= pd.read_sql_query(query, conn)
result

,ORIGIN,ORIGIN_CITY,Departure_Delay_Percentage
0,CDB,"Cold Bay, AK",71.429
1,OGD,"Ogden, UT",55.556
2,HGR,"Hagerstown, MD",54.945
3,PPG,"Pago Pago, TT",54.839
4,MDW,"Chicago, IL",51.674
5,ADK,"Adak Island, AK",51.220
6,HTS,"Ashland, WV",50.855
7,DAL,"Dallas, TX",50.485
8,SCK,"Stockton, CA",50.000
9,HOU,"Houston, TX",48.784


- This query identifies the top 10 airports with the highest departure delay percentages.
- It focuses on airports contributing significantly to delays.

In [ ]:
result.to_csv('result6.csv', index=False)

# **Most and Least Punctual Airports**

In [ ]:
query = '''
SELECT DEST,
       DEST_CITY,
       ROUND((COUNT(CASE WHEN ARR_DELAY <= 0 THEN 1 END) * 100.0 / COUNT(*)),3) as On_Time_Percentage
FROM flight_data
GROUP BY DEST
'''
result= pd.read_sql_query(query, conn)
result

,DEST,DEST_CITY,On_Time_Percentage
0,ABE,"Allentown/Bethlehem/Easton, PA",62.378
1,ABI,"Abilene, TX",66.914
2,ABQ,"Albuquerque, NM",61.604
3,ABR,"Aberdeen, SD",80.000
4,ABY,"Albany, GA",75.286
...,...,...,...
375,XNA,"Fayetteville, AR",64.597
376,XWA,"Williston, ND",68.709
377,YAK,"Yakutat, AK",69.318
378,YKM,"Yakima, WA",61.871


- This query determines the punctuality performance of destination airports by calculating the percentage of on-time arrivals.
- It can help identify the most and least punctual airports.


In [ ]:
result.to_csv('result7.csv', index=False)

# **On-Time Performance by Carrier**

In [ ]:
query = '''
SELECT DOT_CODE AS UNIQUE_CARRIER,
        AIRLINE,
       ROUND((COUNT(CASE WHEN DEP_DELAY <= 0 THEN 1 END) * 100.0 / COUNT(*)),3) as On_Time_Percentage
FROM flight_data
GROUP BY UNIQUE_CARRIER
ORDER BY On_Time_Percentage DESC;
'''
result= pd.read_sql_query(query, conn)
result

,UNIQUE_CARRIER,AIRLINE,On_Time_Percentage
0,20363,Endeavor Air Inc.,78.540
1,20452,Republic Airline,76.428
2,20304,SkyWest Airlines Inc.,73.551
3,20398,Envoy Air,71.225
4,20397,PSA Airlines Inc.,70.421
5,20366,ExpressJet Airlines LLC d/b/a aha!,69.935
6,20378,Mesa Airlines Inc.,69.595
7,19687,Horizon Air,69.361
8,19790,Delta Air Lines Inc.,68.522
9,19930,Alaska Airlines Inc.,66.351


- This query evaluates airline performance by calculating the cancellation percentage for each carrier.
- It helps to rank airlines based on their reliability.


In [ ]:
result.to_csv('result8.csv', index=False)

# **Cancellations by Airline**

In [ ]:
query = '''
SELECT DOT_CODE AS UNIQUE_CARRIER,
        AIRLINE,
       ROUND((COUNT(CASE WHEN CANCELLED=1 THEN 1 END) * 100.0 / COUNT(*)),3) as Cancellation_Percentage
FROM flight_data
GROUP BY UNIQUE_CARRIER
ORDER BY Cancellation_Percentage DESC;
'''
result= pd.read_sql_query(query, conn)
result

,UNIQUE_CARRIER,AIRLINE,Cancellation_Percentage
0,20366,ExpressJet Airlines LLC d/b/a aha!,5.565
1,20368,Allegiant Air,4.519
2,20378,Mesa Airlines Inc.,3.650
3,19393,Southwest Airlines Co.,3.377
4,20452,Republic Airline,3.247
5,20397,PSA Airlines Inc.,3.084
6,20398,Envoy Air,2.996
7,19805,American Airlines Inc.,2.847
8,20409,JetBlue Airways,2.693
9,20436,Frontier Airlines Inc.,2.584


- This query calculates the cancellation percentage for each airline, ranking them to highlight the carriers with the highest and lowest cancellation rates.

In [ ]:
result.to_csv('result9.csv', index=False)

# **Top Delayed Flight Routes**

In [ ]:
query = '''
SELECT ORIGIN || '-' || DEST as Route,
       ROUND((COUNT(CASE WHEN DEP_DELAY > 0 THEN 1 END) * 100.0 / COUNT(*)),3) as Departure_Delay_Percentage,
       ROUND((COUNT(CASE WHEN ARR_DELAY > 0 THEN 1 END) * 100.0 / COUNT(*)),3) as Arrival_Delay_Percentage
FROM flight_data
GROUP BY Route
ORDER BY Departure_Delay_Percentage DESC
LIMIT 10;
'''
result= pd.read_sql_query(query, conn)
result

,Route,Departure_Delay_Percentage,Arrival_Delay_Percentage
0,VPS-TOL,100.0,100.000
1,TWF-SUN,100.0,0.000
2,TWF-SFO,100.0,100.000
3,TPA-ORF,100.0,0.000
4,TOL-VPS,100.0,50.000
5,SMF-BNA,100.0,83.333
6,SJU-CVG,100.0,100.000
7,SJC-RDU,100.0,0.000
8,SEA-HOU,100.0,71.429
9,SDF-SFO,100.0,100.000


- This query identifies the top 10 flight routes with the highest percentages of departure and arrival delays, highlighting problematic routes.


In [ ]:
result.to_csv('result10.csv', index=False)

# **Airline Delay Ranking**

In [ ]:
query = '''
SELECT DOT_CODE AS UNIQUE_CARRIER,
        AIRLINE,
        AVG(DEP_DELAY) as Average_Departure_Delay,
        AVG(ARR_DELAY) as Average_Arrival_Delay
FROM flight_data
GROUP BY UNIQUE_CARRIER
ORDER BY Average_Departure_Delay DESC;
'''
result= pd.read_sql_query(query, conn)
result

,UNIQUE_CARRIER,AIRLINE,Average_Departure_Delay,Average_Arrival_Delay
0,20409,JetBlue Airways,18.322555,12.276125
1,20436,Frontier Airlines Inc.,16.033574,11.100429
2,20368,Allegiant Air,13.907797,13.284601
3,20416,Spirit Air Lines,12.981880,8.029485
4,20366,ExpressJet Airlines LLC d/b/a aha!,12.774462,10.031976
5,19805,American Airlines Inc.,12.609895,6.661229
6,20378,Mesa Airlines Inc.,12.277383,7.309810
7,19977,United Air Lines Inc.,11.217032,5.035985
8,19393,Southwest Airlines Co.,10.816957,3.269796
9,20304,SkyWest Airlines Inc.,9.458294,4.023311


- This query ranks airlines based on their average departure and arrival delays, providing a performance comparison among carriers.


In [ ]:
result.to_csv('result11.csv', index=False)

# **Impact of Taxi Time on Delays**

In [ ]:
query = '''
SELECT DEST,
       DEST_CITY,
       AVG(TAXI_OUT) as Average_Taxi_Out_Time,
       AVG(TAXI_IN) as Average_Taxi_In_Time,
       AVG(DEP_DELAY) as Average_Departure_Delay
FROM flight_data
GROUP BY DEST
'''
result= pd.read_sql_query(query, conn)
result

,DEST,DEST_CITY,Average_Taxi_Out_Time,Average_Taxi_In_Time,Average_Departure_Delay
0,ABE,"Allentown/Bethlehem/Easton, PA",19.159355,4.933367,10.189107
1,ABI,"Abilene, TX",20.527748,3.782372,4.020675
2,ABQ,"Albuquerque, NM",15.646480,5.438868,9.986521
3,ABR,"Aberdeen, SD",15.735385,4.324074,4.061538
4,ABY,"Albany, GA",17.337931,3.986207,4.498851
...,...,...,...,...,...
375,XNA,"Fayetteville, AR",20.150566,6.313350,8.367998
376,XWA,"Williston, ND",18.222973,5.664414,14.078829
377,YAK,"Yakutat, AK",13.416910,4.321321,-1.583090
378,YKM,"Yakima, WA",11.882353,4.139706,5.463235


- This query evaluates the impact of taxi-in and taxi-out times on departure delays at destination airports.
- It offers insights into operational efficiency.


In [ ]:
result.to_csv('result12.csv', index=False)

# **Delay Impact Analysis Using Window Functions**

In [ ]:
query = '''
SELECT FL_DATE, FL_NUMBER, AIRLINE, ORIGIN, DEST,
       DEP_DELAY, ARR_DELAY,
       RANK() OVER (ORDER BY DEP_DELAY DESC) AS Dep_Delay_Rank,
       RANK() OVER (ORDER BY ARR_DELAY DESC) AS Arr_Delay_Rank
FROM flight_data
ORDER BY DEP_DELAY DESC
LIMIT 5;
'''
result= pd.read_sql_query(query, conn)
result

,FL_DATE,FL_NUMBER,AIRLINE,ORIGIN,DEST,DEP_DELAY,ARR_DELAY,Dep_Delay_Rank,Arr_Delay_Rank
0,2022-03-26,2854,American Airlines Inc.,DSM,CLT,2966.0,2934.0,1,1
1,2023-08-25,2560,American Airlines Inc.,VPS,DFW,2938.0,2913.0,2,2
2,2023-07-11,1806,American Airlines Inc.,TPA,PHL,2905.0,2912.0,3,3
3,2022-11-10,2007,American Airlines Inc.,STL,ORD,2903.0,2911.0,4,4
4,2023-05-28,2884,American Airlines Inc.,BFL,PHX,2895.0,2900.0,5,5


- This query uses window functions to rank flights based on their departure and arrival delays, identifying the most delayed flights.


In [ ]:
result.to_csv('result13.csv', index=False)

# **Airport Connectivity and Performance**

In [ ]:
query = '''
SELECT ORIGIN, COUNT(DISTINCT DEST) AS Unique_Routes,
       AVG(DEP_DELAY) AS Avg_Dep_Delay,
       AVG(ARR_DELAY) AS Avg_Arr_Delay
FROM flight_data
GROUP BY ORIGIN
ORDER BY Unique_Routes DESC;

'''
result= pd.read_sql_query(query, conn)
result

,ORIGIN,Unique_Routes,Avg_Dep_Delay,Avg_Arr_Delay
0,ORD,199,11.665374,6.062906
1,DFW,199,12.680089,8.376381
2,DEN,199,13.421166,7.380798
3,ATL,173,8.489554,2.587002
4,CLT,154,8.789748,4.365559
...,...,...,...,...
375,ALO,1,3.166667,0.392857
376,ADQ,1,-2.849858,-0.093484
377,ACT,1,8.932847,6.935673
378,ABY,1,2.856148,-2.437209


- This query analyzes airport connectivity by counting unique routes and evaluates performance using average departure and arrival delays.


In [ ]:
result.to_csv('result14.csv', index=False)

# **Cluster Analysis of Delay Reasons**

In [ ]:
query = '''
SELECT AIRLINE, ORIGIN, DEST,
       COUNT(CASE WHEN DELAY_DUE_CARRIER > 0 THEN 1 END) AS Carrier_Delays,
       COUNT(CASE WHEN DELAY_DUE_WEATHER > 0 THEN 1 END) AS Weather_Delays,
       COUNT(CASE WHEN DELAY_DUE_NAS > 0 THEN 1 END) AS NAS_Delays,
       COUNT(CASE WHEN DELAY_DUE_SECURITY > 0 THEN 1 END) AS Security_Delays,
       COUNT(CASE WHEN DELAY_DUE_LATE_AIRCRAFT > 0 THEN 1 END) AS Late_Aircraft_Delays,
       COUNT(*) AS Total_Flights
FROM flight_data
GROUP BY AIRLINE, ORIGIN, DEST
ORDER BY Total_Flights DESC;

'''
result= pd.read_sql_query(query, conn)
result

,AIRLINE,ORIGIN,DEST,Carrier_Delays,Weather_Delays,NAS_Delays,Security_Delays,Late_Aircraft_Delays,Total_Flights
0,Hawaiian Airlines Inc.,OGG,HNL,517,41,105,2,522,3577
1,Hawaiian Airlines Inc.,HNL,OGG,368,27,31,1,221,3496
2,Alaska Airlines Inc.,SEA,ANC,277,15,353,2,173,2474
3,Hawaiian Airlines Inc.,HNL,KOA,242,19,10,2,157,2467
4,Hawaiian Airlines Inc.,LIH,HNL,239,28,75,0,294,2459
...,...,...,...,...,...,...,...,...,...
17814,United Air Lines Inc.,SJC,LAS,0,0,0,0,1,1
17815,United Air Lines Inc.,SMF,LAX,0,0,1,0,0,1
17816,United Air Lines Inc.,SRQ,IAH,0,0,0,0,0,1
17817,United Air Lines Inc.,TVC,EWR,0,0,0,0,0,1


- This query analyzes delay reasons by categorizing them into carrier, weather, NAS (National Airspace System), security, and late aircraft delays.
- It helps identify patterns and key contributors to delays.


In [ ]:
result.to_csv('result15.csv', index=False)